In [ ]:
# using the saturation values and hue the values we are able to segment the leafs, create a fucntion to make to do the segmentation
def segment_image(image):
    # convert the image to hue 
    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # extract the green colour from the hue
    lower_mask_hue = img_hsv [:,:,0] >= 25
    upper_mask_hue = img_hsv [:,:,0] <= 75
    saturation_lower = img_hsv [:,:,1] >= 40
    saturation_upper = img_hsv[:,:,1] <= 255
    value_lower = img_hsv[:,:,2]>= 50
    value_upper  = img_hsv[:,:,2]<=255
    mask = lower_mask_hue*upper_mask_hue*saturation_lower*saturation_upper*value_lower*value_upper
    red = img_hsv[:,:,0]*mask
    green = img_hsv[:,:,1]*mask
    blue = img_hsv[:,:,2]*mask
    mask = np.dstack((red,green,blue))
    mask = cv2.normalize(mask, None, alpha=0,beta =200, norm_type=cv2.NORM_MINMAX)
    return mask

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
from glob import glob
# Input data files are available in the read-only "../input/" directory
def load_images(path):
    images = []
    labels = []
    for dirname, folders,_ in os.walk(path):
        for folder in folders:
            path = os.path.join(dirname,folder)
            for image_path in glob(os.path.join(path, "*.png")):
                image = cv2.imread(image_path, cv2.IMREAD_COLOR)
                image = segment_image(image)
                image = cv2.resize(image, (224,224), interpolation = cv2.INTER_AREA)
                images.append(image)
                labels.append(folder)
    return images,labels


In [ ]:
from keras.utils import np_utils
from sklearn import preprocessing
images,labels = load_images('/kaggle/input/plant-seedlings-classification/train')
labels = pd.DataFrame(labels)
le = preprocessing.LabelEncoder()
le.fit(labels[0])
encodeTrainLabels = le.transform(labels[0])
classes = np_utils.to_categorical(encodeTrainLabels)
images = np.asarray(images)
classes = np.asarray(classes).astype('int').reshape((4750,12))

In [ ]:
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(images, classes, 
                                                test_size=0.2, random_state=2022, 
                                                stratify = classes)

In [ ]:
import tensorflow as tf
input_model = tf.keras.layers.Input(shape = (224, 224, 3), name = 'image_input')

# main model with Incpetion - ResNet - v2 layers
# omit the top layers as we are adding custom layers
# use transfer learning, with weights from Imagenet trained model
main_model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(include_top = False, weights = 'imagenet')(input_model)

# flatten model to get appropriate dimensions
flattened_model = tf.keras.layers.Flatten()(main_model)

# add custom dropout and dense layers
dropout_1 = tf.keras.layers.Dropout(0.5)(flattened_model)
dense_1 = tf.keras.layers.Dense(128, activation = 'relu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_1)
dropout_2 = tf.keras.layers.Dropout(0.5)(dense_1)

# output of model
output_model = tf.keras.layers.Dense(12, activation = "softmax", activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_2)

model = tf.keras.models.Model(input_model,  output_model)

# use Adam optimizer with model
optimizer = tf.keras.optimizers.Adam(lr = 5e-4, beta_1 = 0.9, beta_2 = 0.999)

# use categorical crossentropy loss since classification task
model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rotation_range=180,  # randomly rotate images in the range
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally
        height_shift_range=0.1,  # randomly shift images vertically 
        horizontal_flip=True,  # randomly flip images horizontally
        vertical_flip=True  # randomly flip images vertically
    )  
datagen.fit(trainX)

In [ ]:
def callbacks():
    
    # save best model regularly
    save_best_model = tf.keras.callbacks.ModelCheckpoint(filepath = 'incepresnet',
        monitor = 'val_acc', save_best_only = True, verbose = 1)

    # reduce learning rate when it stops decreasing
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'loss', factor = 0.4,
                              patience = 3, min_lr = 1e-10, verbose = 1, cooldown = 1)
    
    # stop training early if no further improvement
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'loss', min_delta = 1e-2, patience = 8, verbose = 1,
        mode = 'min', baseline = None, restore_best_weights = True
    )

    return [save_best_model, reduce_lr, early_stopping]

In [ ]:
incep_resnet = model.fit(datagen.flow(trainX,trainY),validation_data=(testX,testY), steps_per_epoch = 110, epochs =15,callbacks = callbacks())

In [ ]:
# testing

images_test = []
testId = []
for image_path in glob(os.path.join("../input/plant-seedlings-classification/test", "*.png")):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = segment_image(image)
    image = cv2.resize(image, (224,224), interpolation = cv2.INTER_AREA)
    images_test.append(image)
    testId.append(image_path.split('/')[-1])

In [ ]:
images_test = np.asarray(images_test)
prediction = model.predict(images_test)
pred_number= np.argmax(prediction, axis=1)
predStr = le.classes_[pred_number]
res = {'file': testId, 'species': predStr}
res = pd.DataFrame(res)
res.to_csv("res.csv", index=False)